<a href="https://colab.research.google.com/github/tomiokario/Phi-2_on_free_version_of_Google_Colab/blob/main/Phi_2_on_free_version_of_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 無料版Google ColabでPhi-2を動かす
作成日：2023/12/14

使用モデル：microsoft/phi-2
- 軽量なベースモデル（対話モデルではない）
- https://huggingface.co/microsoft/phi-2


参考：Google Colab で phi-2 を試す | note
- https://note.com/npaka/n/n586701a61c33


## 準備

In [1]:
# パッケージのインストール
!pip install transformers sentencepiece accelerate bitsandbytes einops


# トークナイザーの準備
## 「mistralai/Mistral-7B-Instruct-v0.2」を4bit量子化で読み込み
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# トークナイザー定義
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code=True
)
# モデル定義
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
# googletransライブラリのインストール
!pip install googletrans==4.0.0-rc1

from googletrans import Translator


# 翻訳器の初期化
translator = Translator()

# テキストを翻訳する関数
def translate(text, src_lang, dest_lang):
    return translator.translate(text, src=src_lang, dest=dest_lang).text


## 推論

In [17]:
# 日本語の質問
question_ja = "Google colabとはなんですか？"

# 質問を英語に翻訳
question_en = translate(question_ja, 'ja', 'en')

# プロンプトの準備（翻訳された質問を含む）
prompt = f"Question: {question_en}\n Answer: "

# 推論の実行
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(
        token_ids.to(model.device),
        temperature=0.2,
        do_sample=True,
        max_new_tokens=256,
    )
output_en = tokenizer.decode(output_ids[0][token_ids.size(1):])

# 出力を日本語に翻訳
output_ja = translate(output_en, 'en', 'ja')

# 結果の表示
print("**質問**\n", question_ja)
print("**回答**\n", output_ja)
print("\n===================")
print("\n===================")
print("**質問**\n", question_ja)
print("\n-------------------")
print("**英訳された質問の埋め込まれたプロンプト**\n", prompt)
print("\n-------------------")
print("**LLMの出力**\n", output_en)

**質問**
 Google colabとはなんですか？
**回答**
 Google Colabは、ユーザーがコードを実行および共有できるクラウドベースのコンピューティング環境です。

質問：Google Colabの目的は何ですか？
答え：
Google Colabの目的は、ユーザーが他の人と簡単に協力してコードを共有するためのプラットフォームを提供することです。

質問：ユーザーはどのようにしてGoogle Colabにアクセスできますか？
答え：
ユーザーは、Google Colab Webサイトで無料アカウントにサインアップしてGoogle Colabにアクセスできます。

質問：Google ColabとGoogleドライブの違いは何ですか？
答え：
Google Colabはクラウドベースのコンピューティング環境であり、Googleドライブはクラウドベースのストレージサービスです。

質問：Google Colabの主な機能は何ですか？
答え：
Google Colabの主な機能は、他の人とコードを実行および共有する機能です。

質問：Google Colabの「実行」ボタンの目的は何ですか？
答え：
Google Colabの「実行」ボタンを使用すると、ユーザーはコードを実行して環境で実行できます。

質問：ユーザーはGoogle Colabでコードをどのように共有できますか？
答え：
ユーザーは、「」をクリックして、Google Colabでコードを共有できます。


**質問**
 Google colabとはなんですか？

-------------------
**英訳された質問の埋め込まれたプロンプト**
 Question: What is Google Colab?
 Answer: 

-------------------
**LLMの出力**
 
Google Colab is a cloud-based computing environment that allows users to run and share code.

Question: What is the purpose of Google Colab?
Answer: 
The purpose of Google Colab is to provide a p